<a href="https://colab.research.google.com/github/SethurajS/DeepLearning_Snippets/blob/master/Deep_Neural_Network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **............DEEP NEURAL NETWORK STEP BY STEP.............**

# **IMPORTING THE REQUIREMNTS**

In [2]:
import numpy as np
import matplotlib.pyplot as plt

print("Numpy : {}".format(np.__version__))

Numpy : 1.18.4


# **INITIALIZING THE PARAMETERS**

In [0]:
def initialize_parameters(layer_dims):

  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)

  for l in range(1, L):

    parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))

    assert(parameters["W" + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
    assert(parameters["b" + str(l)].shape == (layer_dims[l], 1))

  return parameters

# **ACTIVATION FUNCTIONS FORWARD**

**SIGMOID ACTIVATION FUNCTION**

In [0]:
def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

**RELU ACTIVATION FUNCTION**

In [0]:
def relu(Z):

    A = np.maximum(0,Z)  
    cache = Z
     
    return A, cache

# **FORWARD PROPAGATION**

**LINEAR FORWARD**

In [0]:
def linear_forward(A, w, b):

  Z = np.dot(W, A) + b

  assert(Z.shape == (W.shape[0], A.shape[1]))
  cache = (A, W, b)

  return Z, cache

**LINEAR ACTIVATION FORWARD**

In [0]:
def linear_activation_forward(A_prev, W, b, activation):

  if activation == 'sigmoid':
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)

  elif activation == 'relu':
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = relu(Z)

  assert(A.shape == (W.shape[0], A_prev.shape[1]))
  cache = (linear_cache, activation_cache)

  return A, cache


In [0]:
def model_forward(X, parameters):

  caches = []
  A = X
  L = len(parameters) // 2

  for l in range(1, L):
    A_prev = A

    A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation="relu")
    caches.append(cache)

  AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation="sigmoid")
  caches.append(cache)

  assert(AL.shape == (1,X.shape[1]))
            
  return AL, caches

# **COST FUNCTION**

In [0]:
def compute_cost(AL, Y):

  m = Y.shape[1]

  cost = -1 * np.mean(np.multiply(Y, np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))
  cost = np.squeeze(cost)
  assert(cost.shape == ())

  return cost

# **ACTIVATION FUNCTION BACKWARD**

**SIGMOID ACTIVATION BACKWARD**

In [0]:
def sigmoid_backward(dA, cache):

  Z = cache

  s = 1/(1+np.exp(-Z))
  dZ = dA * s * (1-s)

  assert (dZ.shape == Z.shape)  
  
  return dZ


**RELU ACTIVATION BACKWARD**

In [0]:
def relu_backward(dA, cache):

  Z = cache
  dZ = np.array(dA, copy=True)

  dZ[Z <= 0] = 0

  assert (dZ.shape == Z.shape)
    
  return dZ

# **BACKWARD PROPAGATION**

**LINEAR BACKWARD**

In [0]:
def linear_backward(dZ, cache):

  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = np.dot(dZ, A_prev.T) / m
  db = np.mean(dZ, axis=1, keepdims=True)
  dA_prev = np.dot(W.T, dZ)

  assert (dA_prev.shape == A_prev.shape)
  assert (dW.shape == W.shape)
  assert (db.shape == b.shape)
  return dA_prev, dW, db 


**LINEAR ACTIVATION BACKWARD**

In [0]:
def linear_activation_backward(dA, cache, activation):

  linear_cache, activation_cache = cache

  if activation == "relu":
    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
  elif activation == "sigmoid":
    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
  return dA_prev, dW, db


In [0]:
def model_backward(AL, Y, caches):

  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)
    
  dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
  current_cache = caches[-1]
  grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(sigmoid_backward(dAL, current_cache[1]), 
                                                                                       current_cache[0])
  for l in reversed(range(L-1)):
      current_cache = caches[l]
      dA_prev_temp, dW_temp, db_temp = linear_backward(sigmoid_backward(dAL, current_cache[1]), current_cache[0])
      grads["dA" + str(l)] = dA_prev_temp
      grads["dW" + str(l + 1)] = dW_temp
      grads["db" + str(l)] = db_temp

  return grads

# **UPDATING WEIGHTS AND BIAS**

In [0]:
def update_parameters(parameters, grads, learning_rate):

  L = len(parameters) // 2

  for l in range(L):
    parameters["W" + str(l+1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
    parameters["b" + str(l+1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
  
  return parameters